# 03_hyperparameter_tuning (Diabetes)
> Hyperparameters: `learning_rate`, `batch_size`, `hidden_layers`, `units`, `dropout`, `activation`, `optimizer`.


In [ ]:
import os, json, itertools, numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from pathlib import Path
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

DATA = Path('data/processed')
train = pd.read_csv(DATA/'train.csv')
val   = pd.read_csv(DATA/'val.csv')
test  = pd.read_csv(DATA/'test.csv')

X_train, y_train = train.drop(columns=['target']).values, train['target'].values
X_val,   y_val   = val.drop(columns=['target']).values,   val['target'].values
X_test,  y_test  = test.drop(columns=['target']).values,  test['target'].values
n_features = X_train.shape[1]

def make_optimizer(name, lr):
    if name=='adam':   return Adam(lr)
    if name=='sgd':    return SGD(learning_rate=lr, momentum=0.9, nesterov=True)
    if name=='rmsprop':return RMSprop(learning_rate=lr)
    raise ValueError(name)

def make_model(n, hidden_layers=2, units=64, activation='relu', dropout=0.3, use_bn=True, l2=1e-4):
    m = Sequential()
    m.add(Dense(units, activation=activation, input_shape=(n,), kernel_regularizer=regularizers.l2(l2) if l2>0 else None))
    if use_bn: m.add(BatchNormalization())
    if dropout>0: m.add(Dropout(dropout))
    for _ in range(hidden_layers-1):
        m.add(Dense(units, activation=activation, kernel_regularizer=regularizers.l2(l2) if l2>0 else None))
        if use_bn: m.add(BatchNormalization())
        if dropout>0: m.add(Dropout(dropout))
    m.add(Dense(1, activation='sigmoid'))
    return m

grid = {
    'lr':        [1e-3, 5e-4],
    'batch':     [32, 64],
    'layers':    [2, 3],
    'units':     [64, 128],
    'dropout':   [0.2, 0.4],
    'activation':['relu', 'elu'],
    'opt':       ['adam', 'sgd']
}

rows = []
for lr, bs, L, U, dr, act, opt_name in itertools.product(
    grid['lr'], grid['batch'], grid['layers'], grid['units'], grid['dropout'], grid['activation'], grid['opt']
):
    opt = make_optimizer(opt_name, lr)
    model = make_model(n_features, hidden_layers=L, units=U, activation=act, dropout=dr, use_bn=True, l2=1e-4)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    cb = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
    hist = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                     epochs=120, batch_size=bs, verbose=0, callbacks=cb)

    y_prob = model.predict(X_test).ravel()
    y_pred = (y_prob >= 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred)

    rows.append({
        'lr': lr, 'batch': bs, 'layers': L, 'units': U, 'dropout': dr, 'activation': act, 'optimizer': opt_name,
        'test_acc': float(acc),
        'best_epoch': int(np.argmin(hist.history['val_loss'])+1),
    })

results = pd.DataFrame(rows).sort_values('test_acc', ascending=False).reset_index(drop=True)
results.head(10)

In [ ]:
sns.set_theme()
plt.figure(figsize=(10,5)); sns.barplot(data=results, x='optimizer', y='test_acc'); plt.title('Optimizer vs Acc'); plt.show()
plt.figure(figsize=(10,5)); sns.lineplot(data=results, x='units', y='test_acc', hue='layers', marker='o'); plt.title('Units/Layers vs Acc'); plt.show()
plt.figure(figsize=(10,5)); sns.barplot(data=results, x='batch', y='test_acc'); plt.title('Batch Size vs Acc'); plt.show()
plt.figure(figsize=(10,5)); sns.barplot(data=results, x='activation', y='test_acc'); plt.title('Activation vs Acc'); plt.show()

In [ ]:
out = Path('visualizations') / 'comparison_plots' / 'tuning_results_diabetes.csv'
out.parent.mkdir(parents=True, exist_ok=True)
results.to_csv(out, index=False)
out, results.shape

In [ ]:
# --- Save tuning plots to visualizations/comparison_plots ---
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

project_root = Path(os.getcwd()).resolve()
if project_root.name == "notebooks":
    project_root = project_root.parent

CMP_DIR = project_root / "visualizations" / "comparison_plots"
CMP_DIR.mkdir(parents=True, exist_ok=True)

# Re-draw compact versions that we can save
plt.figure(figsize=(8,4))
sns.barplot(data=results, x='optimizer', y='test_acc')
plt.title('Optimizer vs Test Accuracy')
plt.tight_layout()
plt.savefig(CMP_DIR / "optimizer_vs_acc.png", dpi=180)
plt.close()

plt.figure(figsize=(8,4))
sns.lineplot(data=results, x='units', y='test_acc', hue='layers', marker='o')
plt.title('Units & Layers vs Test Accuracy')
plt.tight_layout()
plt.savefig(CMP_DIR / "units_layers_vs_acc.png", dpi=180)
plt.close()

plt.figure(figsize=(8,4))
sns.barplot(data=results, x='batch', y='test_acc')
plt.title('Batch Size vs Test Accuracy')
plt.tight_layout()
plt.savefig(CMP_DIR / "batch_vs_acc.png", dpi=180)
plt.close()

plt.figure(figsize=(8,4))
sns.barplot(data=results, x='activation', y='test_acc')
plt.title('Activation vs Test Accuracy')
plt.tight_layout()
plt.savefig(CMP_DIR / "activation_vs_acc.png", dpi=180)
plt.close()

print("Saved comparison plots to:", CMP_DIR)
